# Import 

In [54]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

# Data load

In [55]:
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [3]:
train = train[:int(len(train)*0.01)]

In [56]:
X_train = train.drop('target', axis=1)
y_train = train['target']

# Train Val split

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=119)

# Scaling

In [58]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test = scaler.transform(test)

# definition DNN

In [23]:
def DNN(add_layer, first_layer_size, layer_size, activation, dropout_rate):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(first_layer_size, activation=activation, input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    for i in range(add_layer):
        model.add(tf.keras.layers.Dense(layer_size, activation=activation))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1))
    return model

# Sherpa Setup

In [24]:
import sherpa
import sherpa.algorithms.bayesian_optimization as bayesian_optimization
import tempfile

# parameter

parameters = [sherpa.Continuous('learning_rate', [1e-4, 1e-2], 'log'),
              sherpa.Choice('batch', [32, 64, 128]),
              sherpa.Discrete('add_layer', [1, 5]),
              sherpa.Choice('first_layer_size', [8, 16, 32, 64]),
              sherpa.Choice('layer_size', [8, 16, 32, 64]),       
              sherpa.Choice('activation', ['relu', 'swish']),
              sherpa.Continuous('dropout_rate', [0, 0.5])]
algorithm = alg = sherpa.algorithms.SuccessiveHalving(r=1, R=9, eta=3, s=0, max_finished_configs=1)
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=False,
                     disable_dashboard=True)

In [25]:
model_dir = tempfile.mkdtemp()

In [26]:
from tensorflow import keras
from sklearn.metrics import mean_absolute_error


for trial in study:
    # Getting number of training epochs
    initial_epoch = {1: 0, 3: 1, 9: 4}[trial.parameters['resource']]
    epochs = trial.parameters['resource'] + initial_epoch

    print("-"*100)
    print(f"Trial:\t{trial.id}\nEpochs:\t{initial_epoch} to {epochs}\nParameters:{trial.parameters}\n")

    if trial.parameters['load_from'] == "":
        print(f"Creating new model for trial {trial.id}...\n")

        # Get hyperparameters
        learning_rate = trial.parameters['learning_rate']
        batch_size = trial.parameters['batch']
        add_layer = trial.parameters['add_layer']
        first_layer_size = trial.parameters['first_layer_size']
        layer_size = trial.parameters['layer_size']
        activation = trial.parameters['activation']
        dropout_rate = trial.parameters['dropout_rate']

        # Create model
        model = DNN(add_layer, first_layer_size, layer_size, activation, dropout_rate)
        model.compile(loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate))
        
    else:
        print(f"Loading model from: ", os.path.join(model_dir, trial.parameters['load_from']), "...\n")

        # Loading model
        model = tf.keras.models.load_model(os.path.join(model_dir, trial.parameters['load_from']))


    # Train model
    for i in range(initial_epoch, epochs):
        with tf.device('/device:GPU:0'):
            model.fit(
        X_train, y_train,
        initial_epoch=i,
        epochs = i+1,
        steps_per_epoch=len(X_train) / batch_size,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        validation_steps=len(X_val) / batch_size,
        shuffle=True)
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)

        print("MAE loss: ", mae)
        study.add_observation(trial=trial, iteration=i,
                              objective=-mae,
                              context={'loss': mae})
        

    study.finalize(trial=trial)
    print(f"Saving model at: ", os.path.join(model_dir, trial.parameters['save_to']))
    model.save(os.path.join(model_dir, trial.parameters['save_to']))

    study.save(model_dir)

----------------------------------------------------------------------------------------------------
Trial:	1
Epochs:	0 to 1
Parameters:{'learning_rate': 0.0025015748328547117, 'batch': 64, 'add_layer': 4, 'first_layer_size': 64, 'layer_size': 64, 'activation': 'swish', 'dropout_rate': 0.07761866886538371, 'resource': 1, 'rung': 0, 'load_from': '', 'save_to': '1'}

Creating new model for trial 1...

147/147 [==============================] - 0s 815us/step
MAE loss:  8.772959657650407
Saving model at:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\1
INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\1\assets


INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\1\assets


----------------------------------------------------------------------------------------------------
Trial:	2
Epochs:	0 to 1
Parameters:{'learning_rate': 0.0031616589003147216, 'batch': 128, 'add_layer': 4, 'first_layer_size': 16, 'layer_size': 8, 'activation': 'relu', 'dropout_rate': 0.29994655900127837, 'resource': 1, 'rung': 0, 'load_from': '', 'save_to': '2'}

Creating new model for trial 2...

147/147 [==============================] - 0s 758us/step
MAE loss:  16.551137286518443
Saving model at:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\2
INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\2\assets


INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\2\assets


----------------------------------------------------------------------------------------------------
Trial:	3
Epochs:	0 to 1
Parameters:{'learning_rate': 0.0009310925304722731, 'batch': 64, 'add_layer': 1, 'first_layer_size': 32, 'layer_size': 8, 'activation': 'swish', 'dropout_rate': 0.49898968912924996, 'resource': 1, 'rung': 0, 'load_from': '', 'save_to': '3'}

Creating new model for trial 3...

147/147 [==============================] - 0s 743us/step
MAE loss:  13.082607614492062
Saving model at:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\3
INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\3\assets


...

INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\10\assets


----------------------------------------------------------------------------------------------------
Trial:	11
Epochs:	1 to 4
Parameters:{'learning_rate': 0.0037032776418304105, 'batch': 32, 'add_layer': 2, 'first_layer_size': 8, 'layer_size': 64, 'activation': 'swish', 'dropout_rate': 0.4189505641667687, 'save_to': '11', 'resource': 3, 'rung': 1, 'load_from': '10'}

Loading model from:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\10 ...

Epoch 2/2
147/147 [==============================] - 0s 813us/step
MAE loss:  9.526866286523997
Epoch 3/3
147/147 [==============================] - 0s 821us/step
MAE loss:  9.236324738665166
Epoch 4/4
147/147 [==============================] - 0s 816us/step
MAE loss:  9.19952284189449
Saving model at:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\11
INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\11\assets


INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\11\assets


----------------------------------------------------------------------------------------------------
Trial:	12
Epochs:	4 to 13
Parameters:{'learning_rate': 0.007696661565658455, 'batch': 32, 'add_layer': 4, 'first_layer_size': 32, 'layer_size': 8, 'activation': 'swish', 'dropout_rate': 0.09623561584607077, 'save_to': '12', 'resource': 9, 'rung': 2, 'load_from': '8'}

Loading model from:  C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\8 ...

Epoch 5/5
147/147 [==============================] - 0s 852us/step
MAE loss:  7.5130105795428275
Epoch 6/6
147/147 [==============================] - 0s 856us/step
MAE loss:  7.398179975482871
Epoch 7/7
147/147 [==============================] - 0s 896us/step
MAE loss:  7.3070362521860766
Epoch 8/8
147/147 [==============================] - 0s 866us/step
MAE loss:  7.258820676134272
Epoch 9/9
147/147 [==============================] - 0s 856us/step
MAE loss:  7.315299849061954
Epoch 10/10
147/147 [==============================] - 0s 886us/step
MAE l

INFO:tensorflow:Assets written to: C:\Users\nehcr\AppData\Local\Temp\tmpbvez4c3u\12\assets


# Serached hyperparameter

* learning rate : 0.005, 0.007
* batch : 64, 32
* add_layer : 3, 4
* first_layer_size : 64, 32
* layer_size : 32, 8

# seed hold

In [48]:
import random

# 시드고정
tf.random.set_seed(42)
random.seed(42)
np.random.seed(42)

# Early stopping

In [60]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=earlystop_patience)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

In [61]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate, verbose = 1):
    model = DNN(3, 64, 32, 'swish', 0.1)

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

In [62]:
epoch = 100
batch = 64
learning_rate = 0.005
earlystop_patience = 20

In [63]:
 # DNN 모델 훈련
DNN_model = train(X_train, y_train, X_val, y_val, f'DNN', epoch,
                    batch, learning_rate)
DNN_model.load_weights(f'./check/DNN-{epoch}-{batch}.h5')

if os.path.exists(f'./model') == False:
    os.mkdir(f'./model')

# 모델 저장
DNN_model.save(f'./model/DNN-{epoch}-{batch}.h5')

Epoch 1/100
66079/66110 [============================>.] - ETA: 0s - loss: 84.7983
Epoch 1: val_loss improved from inf to 67.30554, saving model to ./check\DNN-100-64.h5
66110/66110 [==============================] - 105s 2ms/step - loss: 84.7936 - val_loss: 67.3055
Epoch 2/100
66111/66110 [==============================] - ETA: 0s - loss: 73.1716
Epoch 2: val_loss improved from 67.30554 to 63.93384, saving model to ./check\DNN-100-64.h5
66110/66110 [==============================] - 104s 2ms/step - loss: 73.1716 - val_loss: 63.9338
Epoch 3/100
66082/66110 [============================>.] - ETA: 0s - loss: 71.3407
Epoch 3: val_loss improved from 63.93384 to 63.38778, saving model to ./check\DNN-100-64.h5
66110/66110 [==============================] - 104s 2ms/step - loss: 71.3392 - val_loss: 63.3878
Epoch 4/100
66078/66110 [============================>.] - ETA: 0s - loss: 70.7956
Epoch 4: val_loss improved from 63.38778 to 62.82922, saving model to ./check\DNN-100-64.h5
66110/66110 [=

In [64]:
pred = DNN_model.predict(test)

9102/9102 [==============================] - 6s 637us/step


In [66]:
# 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit.csv", index = False)

sample_submission

,id,target
0,TEST_000000,27.943232
1,TEST_000001,49.377777
2,TEST_000002,55.243835
3,TEST_000003,28.966661
4,TEST_000004,42.692909
...,...,...
291236,TEST_291236,44.354324
291237,TEST_291237,51.567940
291238,TEST_291238,29.182550
291239,TEST_291239,28.380165
